# Using Ray for Scaling Up

Daft's default PyRunner is great for experimentation on your laptop, but when it comes times to running much more computationally expensive jobs that need to take advantage of large scale parallelism, you can run Daft on a [Ray](https://www.ray.io/) cluster instead.

## What is a Ray Cluster, and why do I need it?

Ray is a framework that exposes a Python interface for running distributed computation over a cluster of machines. Daft is built to use Ray as a backend for running dataframe operations, allowing it to scale to huge amounts of data and computation.

However even if you do not have a big cluster to use Ray, you can run Ray locally on your laptop (in which case it would spin up a Ray cluster of just a single machine: your laptop), and using Daft's Ray backend would allow Daft to fully utilize your machine's cores.

## Let's get started!


In [ ]:
!pip install getdaft --pre --extra-index-url https://pypi.anaconda.org/daft-nightly/simple
!pip install Pillow

First, we will run our code without changing the runner. By default, Daft uses the "Python Runner" which runs all processing in a single Python process.

Let's try to download the images from our previous [Text-to-Image Generatation tutorial](https://colab.research.google.com/github/Eventual-Inc/Daft/blob/main/tutorials/text_to_image/text_to_image_generation.ipynb) with the PyRunner.

In [ ]:
import os
import urllib.request

PARQUET_URL = "https://huggingface.co/datasets/ChristophSchuhmann/improved_aesthetics_6.5plus/resolve/main/data/train-00000-of-00001-6f24a7497df494ae.parquet"
PARQUET_PATH = "laion_improved_aesthetics_6_5.parquet"

if not os.path.exists(PARQUET_PATH):
    with open(PARQUET_PATH, "wb") as f:
        response = urllib.request.urlopen(PARQUET_URL)
        f.write(response.read())

We limit the dataset to 160 rows and repartition it into 8 partitions for demonstration purposes. This just means that our data will be divided into 8 approximately equal-sized "chunks".

In [ ]:
from daft import DataFrame, col, udf

parquet_df = DataFrame.read_parquet(PARQUET_PATH).limit(160).repartition(8)
parquet_df.show(3)

## Use the PyRunner to download data from URLs

Now, let's try downloading the data from the URLs with `.url.download()` with the default PyRunner backend!

In [ ]:
%%time

import PIL.Image
import io

@udf(return_type=PIL.Image.Image)
def to_image(bytes_col):
    images = []
    for b in bytes_col:
        if b is not None:
            try:
                images.append(PIL.Image.open(io.BytesIO(b)))
            except:
                images.append(None)
        else:
            images.append(None)
    return images

images_df = parquet_df.with_column("images", to_image(col("URL").url.download()))
images_df_pandas = images_df.to_pandas()

Note how long this took (on Google Colab, it should take approximately 20 seconds).

## Using the RayRunner with a local cluster

Great, now let's use the RayRunner instead and see how we can leverage some parallelism to speed up our workload!

To activate the RayRunner, you can either set environment variables for program execution like so:

```
export DAFT_RUNNER=ray
export DAFT_RAY_ADDRESS=...
```

The `DAFT_RAY_ADDRESS` variable can be left unset to have Daft initialize a default local Ray cluster for you, set to `auto` to automatically detect a Ray cluster running locally, or set to `ray://...` to access a remote Ray cluster.

Alternatively, you can set the configs programatically at the start of your program execution, which is what we will demonstrate here

In [ ]:
# NOTE: TO PROCEED WITH THE TUTORIAL, WE WILL RESTART THE NOTEBOOK RUNTIME
# We do this because we need to reset the Runner that Daft is using. Daft expects any config changes to be performed at program initialization.
exit()

In [ ]:
import daft

RAY_ADDRESS = None
daft.context.set_runner_ray(
    # You may provide Daft with the address to an existing Ray cluster if you have one!
    # If this is not provided, Daft will default to spinning up a single-node Ray cluster consisting of just your current local machine
    address=RAY_ADDRESS,
)

In [ ]:
from daft import DataFrame, col, udf

PARQUET_PATH = "laion_improved_aesthetics_6_5.parquet"
parquet_df = DataFrame.read_parquet(PARQUET_PATH).limit(160).repartition(8)
parquet_df.show(3)

In [ ]:
%%time

import PIL.Image
import io

@udf(return_type=PIL.Image.Image)
def to_image(bytes_col):
    images = []
    for b in bytes_col:
        if b is not None:
            try:
                images.append(PIL.Image.open(io.BytesIO(b)))
            except:
                images.append(None)
        else:
            images.append(None)
    return images

images_df = parquet_df.with_column("images", to_image(col("URL").url.download()))
images_df_pandas = images_df.to_pandas()

With exactly the same code, we were able to achieve a 2x speedup in execution (in Google Colab, this step takes about 10 seconds) - what happened here?

It turns out that our workload is [IO Bound](https://en.wikipedia.org/wiki/I/O_bound) because most of the time is spent waiting for data to be downloaded from the URL.

By default, the `.url.download()` UDF requests `num_cpus=1`. Since our Google Colab machine has 2 CPUs, the RayRunner is able to run two of these UDFs in parallel, hence achieving a 2x increase in throughput!

## Remote Ray Clusters

We have seen that using the RayRunner even locally provides us with some speedup already. However, the real power of distributed computing is in allowing us to access thousands of CPUs and GPUs in the cloud, on a remote Ray cluster.

For example, UDFs that request for a single GPU with `@udf(num_gpus=1)` can run in parallel across hundreds of GPUs on a remote Ray cluster, effortlessly scaling your workloads up to take full advantage of the available hardware.

To run Daft on large clusters, check out [Eventual](https://www.eventualcomputing.com) where you have access to a fully managed platform for running Daft at scale.